In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import torch
from src import models, data, lens, functional
from src.utils import experiment_utils

import logging
from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__

'2.1.2+cu121'

In [3]:
device = "cuda:0"
mt = models.load_model("mamba-3b", device=device, fp16=False)
# mt = models.load_model("gptj", device=device, fp16=True)

2024-01-23 14:49:59 src.models INFO     loading state-spaces/mamba-2.8b-slimpj (device=cuda:0, fp16=False)
2024-01-23 14:49:59 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2024-01-23 14:49:59 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /state-spaces/mamba-2.8b-slimpj/resolve/main/config.json HTTP/1.1" 200 0
2024-01-23 14:50:11 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /state-spaces/mamba-2.8b-slimpj/resolve/main/pytorch_model.bin HTTP/1.1" 302 0


/home/local_arnab/miniconda3/envs/relations/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


2024-01-23 14:50:13 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/gpt-neox-20b/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-01-23 14:50:13 src.models INFO     dtype: torch.float32, device: cuda:0, memory: 10.31 GB


In [4]:
dataset = data.load_dataset()

# relation_names = [r.name for r in dataset.relations]
# relation_options = Menu(choices = relation_names, value = relation_names)
# show(relation_options) # !caution: tested in a juputer-notebook. baukit visualizations are not supported in vscode.

2024-01-23 14:50:13 src.data DEBUG    no paths provided, using default data dir: /home/local_arnab/Codes/relations/notebooks/../data
2024-01-23 14:50:13 src.data DEBUG    /home/local_arnab/Codes/relations/notebooks/../data is directory, globbing for json files...
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/characteristic_gender.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/degree_gender.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_birthplace.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_gender.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/bias/name_religion.json
2024-01-23 14:50:13 src.data DEBUG    found relation 

2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/factual/person_native_language.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/factual/person_occupation.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/factual/person_plays_instrument.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/factual/person_plays_position_in_sport.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/factual/person_plays_pro_sport.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/factual/person_university.json
2024-01-23 14:50:13 src.data DEBUG    found relation file: /home/local_arnab/Codes/relations/notebooks/../data/factual/

In [9]:
relation_name = "country capital city"
relation = dataset.filter(relation_names=[relation_name])[0]
print(f"{relation.name} -- {len(relation.samples)} samples")
print("------------------------------------------------------")

experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
train, test = relation.split(5)
print("\n".join([sample.__str__() for sample in train.samples]))

2024-01-23 14:50:24 src.data DEBUG    filtering to only relations: ['country capital city']
country capital city -- 24 samples
------------------------------------------------------
2024-01-23 14:50:24 src.utils.experiment_utils INFO     setting all seeds to 12345
China -> Beijing
Japan -> Tokyo
Italy -> Rome
Brazil -> Bras\u00edlia
Turkey -> Ankara


In [10]:
################### hparams ###################
layer = 20
beta = 8
###############################################

In [11]:
len(train.samples)

5

In [12]:
hasattr(mt.model, "backbone")

False

In [13]:
from src.operators import JacobianIclMeanEstimator

estimator = JacobianIclMeanEstimator(
    mt = mt, 
    h_layer = layer,
    beta = beta
)
operator = estimator(
    relation.set(
        samples=train.samples, 
    )
)

2024-01-23 14:50:29 src.operators WARNING  relation has > 1 prompt_templates, will use first (The capital city of {} is)
2024-01-23 14:50:29 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of China is
2024-01-23 14:50:29 src.operators DEBUG    note that subject=China, h_index=26


h.shape=torch.Size([2560]) | h.requires_grad=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

2024-01-23 15:22:59 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Japan is
2024-01-23 15:22:59 src.operators DEBUG    note that subject=Japan, h_index=26
h.shape=torch.Size([2560]) | h.requires_grad=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

...

  0%|          | 0/2560 [00:00<?, ?it/s]

2024-01-23 16:27:24 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of China is Beijing
The capital city of China is Beijing
The capital city of Brazil is
2024-01-23 16:27:24 src.operators DEBUG    note that subject=Brazil, h_index=21
h.shape=torch.Size([2560]) | h.requires_grad=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

2024-01-23 16:51:54 src.operators DEBUG    estimating J for prompt:
<|endoftext|> The capital city of Brazil is Bras\u00edlia
The capital city of Italy is Rome
The capital city of Turkey is
2024-01-23 16:51:54 src.operators DEBUG    note that subject=Turkey, h_index=26
h.shape=torch.Size([2560]) | h.requires_grad=True
Calculating Jacobians ...


  0%|          | 0/2560 [00:00<?, ?it/s]

In [14]:
operator.metadata

{'Jh': [tensor([-0.2400,  0.0619, -0.2536,  ..., -0.4082,  0.5569, -0.0195]),
  tensor([-1.1018, -0.2701,  0.1442,  ..., -0.6606,  0.7197,  0.0544]),
  tensor([-0.7632,  0.0086,  0.2646,  ..., -0.2043,  0.7461,  0.0945]),
  tensor([-0.9434,  0.5143, -0.0035,  ..., -0.3035, -0.2629, -0.0782]),
  tensor([ 0.2084,  0.0937, -0.1259,  ...,  0.1322,  0.0302,  0.1695])],
 '|w|': [196.27151489257812,
  197.49301147460938,
  179.90147399902344,
  237.5647735595703,
  187.0806427001953],
 '|b|': [508.92474365234375,
  512.164306640625,
  490.67828369140625,
  602.1090087890625,
  557.1630249023438]}

In [15]:
from src.lens import logit_lens
from src import models

logit_lens(mt = mt, h = operator.metadata["Jh"][0].to(models.determine_device(mt)) + operator.bias)

([('\n', 46.414),
  (' ', 45.944),
  (' _____', 45.884),
  (' the', 45.585),
  ('...', 45.395),
  ('?', 45.115),
  (' (', 45.008),
  ('...', 44.947),
  (' ___', 44.846),
  (' \\', 44.832)],
 {})

In [16]:
from src.functional import predict_next_token

predict_next_token(
    mt = mt, 
    prompt = mt.tokenizer.eos_token + " The capital of {} is".format("France"),
)

[[PredictedToken(token=' Paris', prob=0.34679529070854187),
  PredictedToken(token=' a', prob=0.07820755988359451),
  PredictedToken(token=' the', prob=0.059068627655506134),
  PredictedToken(token=' located', prob=0.052434612065553665),
  PredictedToken(token=' also', prob=0.025281159207224846)]]

# Checking $faithfulness$

In [17]:
test = functional.filter_relation_samples_based_on_provided_fewshots(
    mt=mt, test_relation=test, prompt_template=operator.prompt_template, batch_size=4
)

2024-01-23 17:25:01 src.functional DEBUG    filtering for knowns using prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of {} is"


2024-01-23 17:25:02 src.functional DEBUG    sample.subject='South Korea', sample.object='Seoul', predicted=' Seoul' (p=0.960), known=(✓)
2024-01-23 17:25:02 src.functional DEBUG    sample.subject='Colombia', sample.object='Bogot\\u00e1', predicted=' Bog' (p=0.971), known=(✓)
2024-01-23 17:25:02 src.functional DEBUG    sample.subject='Saudi Arabia', sample.object='Riyadh', predicted=' R' (p=0.739), known=(✓)
2024-01-23 17:25:02 src.functional DEBUG    sample.subject='France', sample.object='Paris', predicted=' Paris' (p=0.984), known=(✓)
2024-01-23 17:25:02 src.functional DEBUG    sample.subject='Mexico', sample.object='Mexico City', predicted=' Mexico' (p=0.956), known=(✓)
2024-01-23 17:25:02 src.functional DEBUG    sample.subject='Pakistan', sample.object='Islamabad', predicted=' Islam' (p=0.924), known=(✓)
2024-01-23 17:25:02 src.functional DEBUG    sample.subject='Argentina', sample.object='Buenos Aires', predicted=' Buenos' (p=0.940), known=(✓)
2024-01-23 17:25:02 src.functional DE

In [25]:
sample = [s for s in test.samples if s.subject == "Pakistan"][0]
print(sample)
operator(subject = sample.subject).predictions

Pakistan -> Islamabad
2024-01-23 17:29:05 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Pakistan is"


[PredictedToken(token=' Pakistan', prob=0.6983874440193176),
 PredictedToken(token=' Islam', prob=0.24579568207263947),
 PredictedToken(token=' Pakistani', prob=0.049618929624557495),
 PredictedToken(token=' Lah', prob=0.0012760799145326018),
 PredictedToken(token=' Kar', prob=0.0005116525571793318)]

In [26]:
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject],
    h_layer= operator.h_layer,
)
h = hs_and_zs.h_by_subj[sample.subject]

## Approximating LM computation $F$ as an affine transformation

### $$ F(\mathbf{s}, c_r) \approx \beta \, W_r \mathbf{s} + b_r $$

In [28]:
z = 5 * (operator.weight @ h) + operator.bias

lens.logit_lens(
    mt = mt,
    h = z,
    get_proba = True
)

([(' Islam', 0.507),
  (' Pakistan', 0.333),
  (' Pakistani', 0.01),
  (' Baghdad', 0.009),
  (' Kar', 0.007),
  (' Jak', 0.005),
  ('\n', 0.004),
  (' _____', 0.004),
  (' ___', 0.004),
  (' _', 0.004)],
 {})

In [29]:
correct = 0
wrong = 0
for sample in test.samples:
    predictions = operator(subject = sample.subject).predictions
    known_flag = functional.is_nontrivial_prefix(
        prediction=predictions[0].token, target=sample.object
    )
    print(f"{sample.subject=}, {sample.object=}, ", end="")
    print(f'predicted="{functional.format_whitespace(predictions[0].token)}", (p={predictions[0].prob}), known=({functional.get_tick_marker(known_flag)})')
    
    correct += known_flag
    wrong += not known_flag
    
faithfulness = correct/(correct + wrong)

print("------------------------------------------------------------")
print(f"Faithfulness (@1) = {faithfulness}")
print("------------------------------------------------------------")

2024-01-23 17:31:05 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Argentina is"


sample.subject='Argentina', sample.object='Buenos Aires', predicted=" Buenos", (p=0.8767764568328857), known=(✓)
2024-01-23 17:31:05 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Australia is"
sample.subject='Australia', sample.object='Canberra', predicted=" Sydney", (p=0.4975793659687042), known=(✗)
2024-01-23 17:31:06 src.operators DEBUG    computing h from prompt "<|endoftext|> The capital city of China is Beijing
The capital city of Japan is Tokyo
The capital city of Italy is Rome
The capital city of Brazil is Bras\u00edlia
The capital city of Turkey is Ankara
The capital city of Canada is"
sample.subject='Canada', sample.object='Ottawa', predicted=" Ottawa", (p=0.6685453057289124), known=(✓)
2024-01-23 17:31:06 src.operators DEBUG    computing h from prompt "<|e

# $causality$

In [30]:
################### hparams ###################
rank = 100
###############################################

In [31]:
experiment_utils.set_seed(12345) # set seed to a constant value for sampling consistency
test_targets = functional.random_edit_targets(test.samples)

2024-01-23 17:31:15 src.utils.experiment_utils INFO     setting all seeds to 12345


## setup

In [32]:
source = test.samples[0]
target = test_targets[source]

f"Changing the mapping ({source}) to ({source.subject} -> {target.object})"

'Changing the mapping (Argentina -> Buenos Aires) to (Argentina -> Riyadh)'

### Calculate $\Delta \mathbf{s}$ such that $\mathbf{s} + \Delta \mathbf{s} \approx \mathbf{s}'$

<p align="center">
    <img align="center" src="causality-crop.png" style="width:80%;"/>
</p>

Under the relation $r =\, $*plays the instrument*, and given the subject $s =\, $*Miles Davis*, the model will predict $o =\, $*trumpet* **(a)**; and given the subject $s' =\, $*Cat Stevens*, the model will now predict $o' =\, $*guiter* **(b)**. 

If the computation from $\mathbf{s}$ to $\mathbf{o}$ is well-approximated by $operator$ parameterized by $W_r$ and $b_r$ **(c)**, then $\Delta{\mathbf{s}}$ **(d)** should tell us the direction of change from $\mathbf{s}$ to $\mathbf{s}'$. Thus, $\tilde{\mathbf{s}}=\mathbf{s}+\Delta\mathbf{s}$ would be an approximation of $\mathbf{s}'$ and patching $\tilde{\mathbf{s}}$ in place of $\mathbf{s}$ should change the prediction to $o'$ = *guitar* 

In [33]:
def get_delta_s(
    operator, 
    source_subject, 
    target_subject,
    rank = 100,
    fix_latent_norm = None, # if set, will fix the norms of z_source and z_target
):
    w_p_inv = functional.low_rank_pinv(
        matrix = operator.weight,
        rank=rank,
    )
    hs_and_zs = functional.compute_hs_and_zs(
        mt = mt,
        prompt_template = operator.prompt_template,
        subjects = [source_subject, target_subject],
        h_layer= operator.h_layer,
        z_layer=-1,
    )

    z_source = hs_and_zs.z_by_subj[source_subject]
    z_target = hs_and_zs.z_by_subj[target_subject]
    
    z_source *= fix_latent_norm / z_source.norm() if fix_latent_norm is not None else 1.0
    z_target *= z_source.norm() / z_target.norm() if fix_latent_norm is not None else 1.0

    delta_s = w_p_inv @  (z_target.squeeze() - z_source.squeeze())

    return delta_s, hs_and_zs

delta_s, hs_and_zs = get_delta_s(
    operator = operator,
    source_subject = source.subject,
    target_subject = target.subject,
    rank = rank
)

In [36]:
import baukit

def get_intervention(h, int_layer, subj_idx):
    def edit_output(output, layer):
        if(layer != int_layer):
            return output
        functional.untuple(output)[:, subj_idx] = h 
        return output
    return edit_output

prompt = operator.prompt_template.format(source.subject)

h_index, inputs = functional.find_subject_token_index(
    mt=mt,
    prompt=prompt,
    subject=source.subject,
)

h_layer, z_layer = models.determine_layer_paths(model = mt, layers = [layer, -1])

with baukit.TraceDict(
    mt.model, layers = [h_layer, z_layer],
    edit_output=get_intervention(
#         h = hs_and_zs.h_by_subj[source.subject],         # let the computation proceed as usual
        h = hs_and_zs.h_by_subj[source.subject] + delta_s, # replace s with s + delta_s
        int_layer = h_layer, 
        subj_idx = h_index
    )
) as traces:
    outputs = mt(
        input_ids = inputs.input_ids,
        attention_mask = inputs.attention_mask,
    )

logits = outputs.logits[0][-1] if hasattr(outputs, "logits") else outputs[0][-1]
lens.interpret_logits(
    mt = mt, 
    logits = logits, 
    get_proba=True
)

[(' R', 0.684),
 (' Me', 0.109),
 (' Ri', 0.023),
 (' Jed', 0.015),
 (' Saudi', 0.013),
 ('\n', 0.009),
 (' Med', 0.006),
 (' J', 0.005),
 (' Mak', 0.005),
 ('...', 0.005)]

## Measuring causality

In [37]:
from src.editors import LowRankPInvEditor

svd = torch.svd(operator.weight.float())
editor = LowRankPInvEditor(
    lre=operator,
    rank=rank,
    svd=svd,
)

In [146]:
# precomputing latents to speed things up
hs_and_zs = functional.compute_hs_and_zs(
    mt = mt,
    prompt_template = operator.prompt_template,
    subjects = [sample.subject for sample in test.samples],
    h_layer= operator.h_layer,
    z_layer=-1,
    batch_size = 2
)

success = 0
fails = 0

for sample in test.samples:
    target = test_targets.get(sample)
    assert target is not None
    edit_result = editor(
        subject = sample.subject,
        target = target.subject
    )
    
    success_flag = functional.is_nontrivial_prefix(
        prediction=edit_result.predicted_tokens[0].token, target=target.object
    )
    
    print(f"Mapping {sample.subject} -> {target.object} | edit result={edit_result.predicted_tokens[0]} | success=({functional.get_tick_marker(success_flag)})")
    
    success += success_flag
    fails += not success_flag
    
causality = success / (success + fails)

print("------------------------------------------------------------")
print(f"Causality (@1) = {causality}")
print("------------------------------------------------------------")

Mapping Argentina -> Riyadh | edit result=' R' (p=0.714) | success=(✓)
Mapping Australia -> Buenos Aires | edit result=' Buenos' (p=0.898) | success=(✓)
Mapping Canada -> Abuja | edit result=' Lag' (p=0.359) | success=(✗)
Mapping Chile -> Lima | edit result=' L' (p=0.952) | success=(✓)
Mapping Colombia -> Berlin | edit result=' Berlin' (p=0.933) | success=(✓)
Mapping Egypt -> Mexico City | edit result=' Mexico' (p=0.966) | success=(✓)
Mapping France -> Riyadh | edit result=' R' (p=0.569) | success=(✓)
Mapping Germany -> Cairo | edit result=' Cairo' (p=0.914) | success=(✓)
Mapping India -> Lima | edit result=' L' (p=0.901) | success=(✓)
Mapping Mexico -> Santiago | edit result=' Santiago' (p=0.800) | success=(✓)
Mapping Nigeria -> Riyadh | edit result=' R' (p=0.694) | success=(✓)
Mapping Pakistan -> New Delhi | edit result=' New' (p=0.806) | success=(✓)
Mapping Peru -> Caracas | edit result=' Car' (p=0.763) | success=(✓)
Mapping Russia -> Cairo | edit result=' Cairo' (p=0.915) | success

In [141]:
from src.functional import mamba_generate

result = mamba_generate(
    mt = mt,
    prompt = ["The capital of France is"]*10,
    edit_config=None
)

In [142]:
result.model_generations

[' Paris.\nThe currency of France is Euro (',
 ' Paris, the second most important city is Marse',
 ' also the city with the most expensive real estate in',
 ' Paris.\nThe capital of France is Paris,',
 ' Paris, the most popular city in the country.',
 ' Paris.\nParis, the city of love,',
 ' Paris. It is the largest city and principal city',
 ' Paris.\nThe capital of France is Paris.',
 " Paris, and the country's largest city is Lyon",
 ' Paris\nThe capital of France is the city of']

In [143]:
result.predicted_tokens

[PredictedToken(token=' Paris', prob=0.5615785121917725),
 PredictedToken(token=' a', prob=0.03813231363892555),
 PredictedToken(token=' the', prob=0.03712513670325279),
 PredictedToken(token=' located', prob=0.0301530621945858),
 PredictedToken(token=' also', prob=0.018496785312891006)]

In [145]:
dir(mamba_generate)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__wrapped__']